In [5]:
import torch
import timm
import pandas as pd
import numpy as np
from pathlib import Path
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from data_loader import build_loaders

In [ ]:
checkpoint_path = "./model_weights/0423_1455_epoch=39_val_f1=0.8880.ckpt"  # 실제 체크포인트 경로
model_name      = "vit_xsmall_patch16_clip_224"
num_classes     = 7
img_size        = 224
batch_size      = 64
num_workers     = 8
device          = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Data 로드
train_loader, val_loader, le = build_loaders(
    data_dir="./data/train",
    test_size=0.2,
    img_size=args.img_size,
    batch_size=args.batch_size,
    num_workers=args.num_workers,
    seed=args.seed,
    mean = (0.38771973, 0.39787053, 0.40713646), # 전역분산
    std = (0.2130759, 0.21581389, 0.22090413),
)